In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv
/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv
/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv


In [57]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from collections import Counter
import re
from sklearn import model_selection
from sklearn import metrics
import lightgbm as lgb


In [58]:
input_dir="/kaggle/input/linking-writing-processes-to-writing-quality"
train_logs=pd.read_csv(f'{input_dir}/train_logs.csv')
test_logs=pd.read_csv(f'{input_dir}/test_logs.csv')
train_scores=pd.read_csv(f'{input_dir}/train_scores.csv')

In [59]:
def getEssays(df):
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    lastIndex = 0
    essaySeries = pd.Series()

    for index, valCount in enumerate(valCountsArr):

        # Indexes down_time at current Id
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]

        # Update the last index
        lastIndex += valCount

        # Where the essay content will be stored
        essayText = ""

        
        # Produces the essay
        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[1] = cursor_position
            # Input[2] = text_change
            
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]
                
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')
                
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                
                
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

            
        # Sets essay at index  
        essaySeries[index] = essayText
     
    
    # Sets essay series index to the ids
    essaySeries.index =  textInputDf['id'].unique()
    
    
    # Returns the essay series
    return essaySeries

In [60]:
train_essays = getEssays(train_logs)
test_essays=getEssays(test_logs)

In [61]:
train_essaysdf = pd.DataFrame({'id': train_essays.index, 'essay': train_essays.values})
test_essaysdf = pd.DataFrame({'id': test_essays.index, 'essay': test_essays.values})

merged_data = train_essaysdf.merge(train_scores, on='id')
print(merged_data)

            id                                              essay  score
0     001519c8  qqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqqq qqq q...    3.5
1     0022f953  qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq...    3.5
2     0042269b  qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...    6.0
3     0059420b  qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...    2.0
4     0075873a  qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq...    4.0
...        ...                                                ...    ...
2466  ffb8c745       qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq...    3.5
2467  ffbef7e5  qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq...    4.0
2468  ffccd6fd  qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...    1.5
2469  ffec5b38  qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq...    5.0
2470  fff05981  qq qqqq qqqqqqq qqqqqqqq qq qqqqqqqqqqq qq qq ...    4.0

[2471 rows x 3 columns]


In [62]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

In [63]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_tokenizer_train = count_vectorizer.fit_transform(merged_data['essay'])
X_tokenizer_test = count_vectorizer.transform(test_essaysdf['essay'])
count_vectorizer.get_feature_names_out() #ADDED
y = merged_data['score']

In [64]:
dense_array_train = X_tokenizer_train.toarray()
dense_array_test = X_tokenizer_test.toarray()

df_train_features = pd.DataFrame(dense_array_train, columns=[f'feature {i}' for i in range(dense_array_train.shape[1])])
df_test_features = pd.DataFrame(dense_array_test, columns=[f'feature {i}' for i in range(dense_array_test.shape[1])])

df_train = pd.concat([df_train, df_train_features], axis=1)
df_test = pd.concat([df_test, df_test_features], axis=1)

In [65]:
df_train_index = train_essaysdf['id']
df_test_index = test_essaysdf['id']
df_train.loc[:, 'id'] = df_train_index
df_test.loc[:, 'id'] = df_test_index
df_train

,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,...,feature 282,feature 283,feature 284,feature 285,feature 286,feature 287,feature 288,feature 289,feature 290,id
0,53,8,12,8,6,8,1,4,4,2,...,0,0,0,0,0,0,0,0,0,001519c8
1,61,8,12,19,11,5,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0022f953
2,64,6,8,8,6,6,4,10,4,4,...,0,0,0,0,0,0,0,0,0,0042269b
3,44,6,7,8,4,6,6,1,1,2,...,0,0,0,0,0,0,0,0,0,0059420b
4,49,3,15,11,9,2,3,3,0,1,...,0,0,0,0,0,0,0,0,0,0075873a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,49,5,4,9,13,7,7,0,1,0,...,0,0,0,0,0,0,0,0,0,ffb8c745
2467,74,16,9,21,10,7,4,4,1,1,...,0,0,0,0,0,0,0,0,0,ffbef7e5
2468,42,5,9,8,7,5,3,1,1,1,...,0,0,0,0,0,0,0,0,0,ffccd6fd
2469,80,5,19,12,11,6,11,2,1,5,...,0,0,0,0,0,0,0,0,0,ffec5b38


In [66]:
train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)
print(train_agg_fe_df)
test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

            id  tmp_down_time_mean  tmp_down_time_std  tmp_down_time_min  \
0     001519c8        8.481808e+05      395112.665961               4526   
1     0022f953        5.188553e+05      384959.404177              30623   
2     0042269b        8.284918e+05      489500.796565               4441   
3     0059420b        7.854830e+05      385205.014399              41395   
4     0075873a        7.133542e+05      405576.409034              78470   
...        ...                 ...                ...                ...   
2466  ffb8c745        7.361019e+05      503882.020411              22467   
2467  ffbef7e5        8.419662e+05      512744.745940              21732   
2468  ffccd6fd        1.229015e+06      514320.848199              23482   
2469  ffec5b38        5.765185e+05      334477.976640              19885   
2470  fff05981        1.076844e+06      581107.759299              39727   

      tmp_down_time_max  tmp_down_time_last  tmp_down_time_first  \
0               180

In [67]:
import torch
from transformers import BertTokenizer, BertModel

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 使用 Preprocessor 类进行文本嵌入
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

train=pd.DataFrame(columns=['bert_embedding'])
test=pd.DataFrame(columns=['bert_embedding'])

for row in train_essays.items():
    text = row[1]
    # 使用分词器对文本进行编码
    input_ids = tokenizer.encode(text, return_tensors='pt',max_length=512, truncation=True).to(device)
    
    # 获取BERT模型的输出
    with torch.no_grad():
        outputs = model(input_ids)
    
    # 提取最后一层的隐藏状态
    last_hidden_states = outputs.last_hidden_state
    sentence_vector = last_hidden_states.mean(dim=1)
    # 在这里可以使用last_hidden_states作为特征进行进一步的处理
    # ...

    # 例如，将嵌入表示添加到DataFrame中
    train.loc[len(train)] = [sentence_vector.cpu().numpy()]
print(train)

for row in test_essays.items():
    text = row[1]
    # 使用分词器对文本进行编码
    input_ids = tokenizer.encode(text, return_tensors='pt',max_length=512, truncation=True).to(device)
    
    # 获取BERT模型的输出
    with torch.no_grad():
        outputs = model(input_ids)
    
    # 提取最后一层的隐藏状态
    last_hidden_states = outputs.last_hidden_state
    sentence_vector = last_hidden_states.mean(dim=1)
    # 在这里可以使用last_hidden_states作为特征进行进一步的处理
    # ...

    # 例如，将嵌入表示添加到DataFrame中
    test.loc[len(test)] = [sentence_vector.cpu().numpy()]

                                         bert_embedding
0     [[0.10270646, 0.31191358, 0.21102765, -0.70029...
1     [[0.08346323, 0.24788868, 0.2624147, -0.646316...
2     [[0.11100255, 0.32315826, 0.19025856, -0.70320...
3     [[0.11526609, 0.32681537, 0.20288798, -0.69565...
4     [[0.015251498, 0.23293838, 0.4429636, -0.66533...
...                                                 ...
2466  [[0.023321021, 0.25657976, 0.36098096, -0.6651...
2467  [[0.030726206, 0.24117437, 0.40864456, -0.6456...
2468  [[0.13183317, 0.39548647, 0.030384887, -0.7385...
2469  [[0.09183662, 0.31349424, 0.22715673, -0.69854...
2470  [[0.10618518, 0.34759587, 0.14362203, -0.77023...

[2471 rows x 1 columns]
123
123
123


In [69]:
train_agg_fe_df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
train_agg_fe_df.columns = ['_'.join(x) for x in train_agg_fe_df.columns]
train_agg_fe_df = train_agg_fe_df.add_prefix("tmp_")
train_agg_fe_df.reset_index(inplace=True)
print(train_agg_fe_df)
test_agg_fe_df = test_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum'])
test_agg_fe_df.columns = ['_'.join(x) for x in test_agg_fe_df.columns]
test_agg_fe_df = test_agg_fe_df.add_prefix("tmp_")
test_agg_fe_df.reset_index(inplace=True)

            id  tmp_down_time_mean  tmp_down_time_std  tmp_down_time_min  \
0     001519c8        8.481808e+05      395112.665961               4526   
1     0022f953        5.188553e+05      384959.404177              30623   
2     0042269b        8.284918e+05      489500.796565               4441   
3     0059420b        7.854830e+05      385205.014399              41395   
4     0075873a        7.133542e+05      405576.409034              78470   
...        ...                 ...                ...                ...   
2466  ffb8c745        7.361019e+05      503882.020411              22467   
2467  ffbef7e5        8.419662e+05      512744.745940              21732   
2468  ffccd6fd        1.229015e+06      514320.848199              23482   
2469  ffec5b38        5.765185e+05      334477.976640              19885   
2470  fff05981        1.076844e+06      581107.759299              39727   

      tmp_down_time_max  tmp_down_time_last  tmp_down_time_first  \
0               180

In [70]:
from collections import defaultdict

class Preprocessor:
    
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        
        self.idf = defaultdict(float)
#         self.gaps = [1, 2]
    
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf

        return ret


    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
        ret = pd.DataFrame(ret)
        cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
        ret.columns = cols

        cnts = ret.sum(1)

        for col in cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf
            
            ret[col] = 1 + np.log(ret[col] / cnts)
            ret[col] *= idf
            
        return ret

    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret


    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        # cursor position shift
        print("Engineering cursor position data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        # word count shift
        print("Engineering word count data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
            ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew']),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew'])
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        # counts
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        # input words
        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']
        
        print("Done!")
        return feats

In [71]:
preprocessor = Preprocessor(seed=42)

print("Engineering features for training data")

other_train_feats = preprocessor.make_feats(train_logs)

print()
print("-"*25)
print("Engineering features for test data")
print("-"*25)
other_test_feats = preprocessor.make_feats(test_logs)

Engineering features for training data
Starting to engineer features
Engineering time data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering cursor position data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering word count data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 100
Engineering statistical summaries for features


100%|██████████| 33/33 [03:10<00:00,  5.78s/it, column=word_count_change100, method=skew]         


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 5212.39it/s]
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

Engineering text change counts data


100%|██████████| 3/3 [00:00<00:00, 24966.10it/s]
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 27962.03it/s]


Engineering input words data
Engineering ratios data
Done!


In [72]:
df_train_all = pd.DataFrame()
df_test_all = pd.DataFrame()

df_train_all = df_train.merge(train_agg_fe_df,on='id')
df_test_all = df_test.merge(test_agg_fe_df,on='id')

In [73]:
train_expanded = train["bert_embedding"].apply(lambda x: pd.Series(x.flatten()))
test_expanded = test["bert_embedding"].apply(lambda x: pd.Series(x.flatten()))

In [74]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [75]:
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', 'sum']

def split_essays_into_sentences(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',str(x)))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.columns.tolist()].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(df):
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

def split_essays_into_paragraphs(df):
    essay_df = df
    essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: str(x).split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

In [76]:
train_sent_df = split_essays_into_sentences(train_essaysdf)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

train_paragraph_df = split_essays_into_paragraphs(train_essaysdf)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_essaysdf))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essaysdf))

train_paragraph_agg_df.loc[:, 'id'] = df_train_index
train_sent_agg_df.loc[:, 'id'] = df_train_index

test_paragraph_agg_df.loc[:, 'id'] = df_test_index
test_sent_agg_df.loc[:, 'id'] = df_test_index

In [77]:
new_train_feats = pd.DataFrame()
new_test_feats = pd.DataFrame()

new_train_feats = train_paragraph_agg_df.merge(df_train_all,on='id')
new_train_feats = new_train_feats.merge(train_sent_agg_df,on='id')

new_test_feats = test_paragraph_agg_df.merge(df_test_all,on='id')
new_test_feats = new_test_feats.merge(test_sent_agg_df,on='id')

train_feats = pd.DataFrame()
test_feats = pd.DataFrame()

train_feats = new_train_feats.merge(other_train_feats,on='id')
test_feats = new_test_feats.merge(other_test_feats,on='id')

In [78]:
data = []

for logs in [train_logs, test_logs]:
    logs['up_time_lagged'] = logs.groupby('id')['up_time'].shift(1).fillna(logs['down_time'])
    logs['time_diff'] = abs(logs['down_time'] - logs['up_time_lagged']) / 1000

    group = logs.groupby('id')['time_diff']
    largest_lantency = group.max()
    smallest_lantency = group.min()
    median_lantency = group.median()
    initial_pause = logs.groupby('id')['down_time'].first() / 1000
    pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
    pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
    pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
    pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
    pauses_3_sec = group.apply(lambda x: (x > 3).sum())

    data.append(pd.DataFrame({
        'id': logs['id'].unique(),
        'largest_lantency': largest_lantency,
        'smallest_lantency': smallest_lantency,
        'median_lantency': median_lantency,
        'initial_pause': initial_pause,
        'pauses_half_sec': pauses_half_sec,
        'pauses_1_sec': pauses_1_sec,
        'pauses_1_half_sec': pauses_1_half_sec,
        'pauses_2_sec': pauses_2_sec,
        'pauses_3_sec': pauses_3_sec,
    }).reset_index(drop=True))

train_eD592674, test_eD592674 = data

train_feats = train_feats.merge(train_eD592674, on='id', how='left')
test_feats = test_feats.merge(test_eD592674, on='id', how='left')
train_feats = train_feats.merge(train_scores, on='id', how='left')

In [79]:
train_feats = pd.concat([train_feats,train_expanded],axis=1)
test_feats = pd.concat([test_feats,test_expanded],axis=1)

In [80]:
train_feats.shape

(2471, 1426)

In [81]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train_feats['score_class'] = le.fit_transform(train_feats['score'])

In [82]:
target_col = ['score']

drop_cols = ['id', 'score_class']
train_cols = list()

train_cols = [col for col in train_feats.columns if col not in target_col + drop_cols]

In [83]:
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()


for col in nan_cols:
    mode_value_train = train_feats[col].mode()[0] 
    train_feats[col].fillna(mode_value_train, inplace=True)
    
for col in test_feats.columns[test_feats.isna().any()].tolist():

    most_frequent_value_train = train_feats[col].mode()[0]
    

    test_feats[col].fillna(most_frequent_value_train, inplace=True)

train_feats.shape, test_feats.shape  

((2471, 1427), (3, 1425))

# **Model**

In [84]:
import gc
import ctypes
import torch
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()
clean_memory()

In [85]:
models_dict = {}
scores = []

test_predict_list = []
best_params = {'boosting_type': 'gbdt', 
               'metric': 'rmse',
               'reg_alpha': 0.003188447814669599, 
               'reg_lambda': 0.0010228604507564066, 
               'colsample_bytree': 0.5420247656839267, 
               'subsample': 0.9778252382803456, 
               'feature_fraction': 0.8,
               'bagging_freq': 1,
               'bagging_fraction': 0.75,
               'learning_rate': 0.01716485155812008, 
               'num_leaves': 19, 
               'min_child_samples': 46,
               'verbosity': -1,
               'random_state': 42,
               'n_estimators': 500,
               'device_type': 'cpu'}

for i in range(5): 
    kf = model_selection.KFold(n_splits=10, random_state=42 + i, shuffle=True)

    oof_valid_preds = np.zeros(train_feats.shape[0], )

    X_test = test_feats[train_cols]


    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):

        print("==-"* 50)
        print("Fold : ", fold)

        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

        print("Trian :", X_train.shape, y_train.shape)
        print("Valid :", X_valid.shape, y_valid.shape)

        params = {
            "objective": "regression",
            "metric": "rmse",
            'random_state': 42,
            "n_estimators" : 12001,
            "verbosity": -1,
            "device_type": "cpu",
            **best_params
        }

        model = lgb.LGBMRegressor(**params)

        early_stopping_callback = lgb.early_stopping(200, first_metric_only=True, verbose=False)
        verbose_callback = lgb.callback.record_evaluation({})

        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],  
                  callbacks=[early_stopping_callback, verbose_callback],
        )

        valid_predict = model.predict(X_valid)
        oof_valid_preds[valid_idx] = valid_predict

        test_predict = model.predict(X_test)
        test_predict_list.append(test_predict)

        score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
        print("Fold RMSE Score : ", score)

        models_dict[f'{fold}_{i}'] = model


    oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_preds, squared=False)
    scores.append(oof_score)
    print("OOF RMSE Score : ", oof_score)

==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  0
Trian : (2223, 1424) (2223, 1)
Valid : (248, 1424) (248, 1)
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5420247656839267 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Fold RMSE Score :  0.5849775813804726
==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-
Fold :  1
Trian : (2224, 1424) (2224, 1)
Valid : (247, 1424) (247, 1)
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=0.9778252382803456 will be ignored. Current value: bagging_fraction=0.75
[Ligh

In [86]:
np.mean(scores)

0.6135433792391919

In [87]:
test_feats['score'] = np.mean(test_predict_list, axis=0)
publiclgbm_pred = test_feats[['id', 'score']]
publiclgbm_pred

,id,score
0,0000aaaa,1.680139
1,2222bbbb,1.536805
2,4444cccc,1.635930


# **Ensemble**

In [88]:
import random
seed=2023
np.random.seed(seed)
random.seed(seed)
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

In [89]:
def make_model():
    
    catboost_params= {
        "iterations": 5000,
        "early_stopping_rounds": 50,
        "depth": 6,
        "loss_function": "RMSE",
        "random_seed": 42,
        "silent": True
    }
    
    lgbm_params= {
        'n_estimators': 1024,
        'learning_rate': 0.005,
        'metric': 'rmse',
        'random_state': 42,
        'force_col_wise': True,
        'verbosity': 0
    }
    
    xgboost_params ={
        "max_depth": 4,
        "learning_rate": 0.1,
        "objective": "reg:squarederror",
        "n_estimators": 1000,  # 修正此处的参数名
        "eval_metric": "rmse",
        "seed": 42
    }
    
    model1 = LGBMRegressor(**lgbm_params)
    
    model2 = CatBoostRegressor(**catboost_params)
    
    model3  = XGBRegressor(**xgboost_params)
    
    models = []
    models.append((model1, 'lgbm'))
    models.append((model2, 'catboost'))
    models.append((model3, 'xgboost'))
    
    return models

In [90]:
num_folds = 10

model_with_scaled_features = ['xgboost']

blending_weights = {
    'lgbm': 0.6,
    'catboost': 0.3,
    'xgboost': 0.1,
}

In [91]:
train_feats.columns = train_feats.columns.astype(str)
test_feats.columns = test_feats.columns.astype(str)
train_cols = [str(col) for col in train_cols]

In [92]:
train_feats

,paragraph_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,...,759,760,761,762,763,764,765,766,767,score_class
0,3,508.000000,134.208793,390,654,390,480,77.485483,435.00,480.0,...,0.543616,0.748779,-1.423322,-0.553759,-0.001282,-0.295785,-0.774561,0.301522,0.085557,6
1,6,278.166667,98.554384,176,462,240,284,40.234659,228.75,261.0,...,0.620695,0.726740,-1.445171,-0.435152,-0.038977,-0.492697,-0.800179,0.328693,0.057892,6
2,6,429.500000,101.087586,296,568,491,296,41.268834,356.75,444.5,...,0.520756,0.750219,-1.417216,-0.586341,-0.024095,-0.249560,-0.753874,0.308836,0.124730,11
3,3,384.000000,56.471232,347,449,347,356,32.603681,351.50,356.0,...,0.532827,0.740490,-1.436533,-0.572211,0.016308,-0.249350,-0.752671,0.312077,0.053063,3
4,5,283.400000,232.336609,23,627,351,23,103.904090,124.00,292.0,...,0.675400,0.604619,-1.481418,-0.379651,-0.113376,-0.490850,-0.772265,0.343489,0.085378,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,4,407.750000,91.790976,301,514,372,301,45.895488,354.25,408.0,...,0.649013,0.675741,-1.480808,-0.407907,-0.113343,-0.412374,-0.763036,0.364216,0.059319,6
2467,6,387.166667,178.941797,144,648,144,228,73.052683,274.00,424.0,...,0.691299,0.637945,-1.519715,-0.354740,-0.129195,-0.512009,-0.793757,0.338959,0.067130,7
2468,3,918.333333,939.787387,327,2002,426,2002,542.586501,376.50,426.0,...,0.478948,0.766021,-1.420998,-0.577128,0.080904,-0.258818,-0.827178,0.186180,0.118300,2
2469,5,509.600000,122.681702,380,672,672,380,54.864925,394.00,540.0,...,0.535551,0.731311,-1.442836,-0.572768,-0.023611,-0.239196,-0.740059,0.334475,0.083982,9


In [93]:
train_feats.replace([np.inf, -np.inf], np.nan, inplace=True)

def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

models_and_errors_dict = {}

for model, model_type in make_model():
    
    oof_pred=np.zeros(train_feats.shape[0])        

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed + num_folds)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):

        print(f'--- Fold #{fold} ---')       
        X_test = test_feats[train_cols]
        # Split data into train and test sets
        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_test, y_test = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]

        X_train_copy, X_test_copy = X_train.copy(), X_test.copy()

        print(f'Training a {model_type} model on fold {fold}')


        if model_type in model_with_scaled_features:

            imputer = SimpleImputer(strategy='mean')
            X_train_imputed = imputer.fit_transform(X_train.copy())
            X_test_imputed = imputer.transform(X_test.copy())

            scaler = MinMaxScaler(feature_range=(-1, 1))
            X_train_scaled = scaler.fit_transform(X_train_imputed)
            X_test_scaled = scaler.transform(X_test_imputed)

            X_train_copy = X_train_scaled
            X_test_copy = X_test_scaled

        if model_type == 'lgb':
            
            early_stopping_callback = LGBMRegressor.early_stopping(200, first_metric_only=True, verbose=False)
            verbose_callback = LGBMRegressor.log_evaluation(100)

            model.fit(X_train_copy, y_train, eval_set=[(X_test_copy, y_test)],  
                      callbacks=[early_stopping_callback, verbose_callback],)
        else:
            model.fit(X_train_copy, y_train)


        y_hat = model.predict(X_test_copy)

        oof_pred[valid_idx]=y_hat

        rmse = RMSE(y_test.values.reshape(-1), y_hat)
        print(f'RMSE: {rmse} on fold {fold}')


        if model_type not in models_and_errors_dict:
            models_and_errors_dict[model_type] = []

        if model_type in model_with_scaled_features:
            models_and_errors_dict[model_type].append((model, rmse, imputer, scaler,oof_pred))
        else:
            models_and_errors_dict[model_type].append((model, rmse, None, None,oof_pred))  

--- Fold #0 ---
Training a lgbm model on fold 0
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.5915866246197972 on fold 0
--- Fold #1 ---
Training a lgbm model on fold 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.6229203543715441 on fold 1
--- Fold #2 ---
Training a lgbm model on fold 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.5799143965420785 on fold 2
--- Fold #3 ---
Training a lgbm model on fold 3
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.6370054652138515 on fold 3
--- Fold #4 ---
Training a lgbm model on fold 4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.6581942026875308 on fold 4
--- Fold #5 ---
Training a lgbm model on fold 5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
RMSE: 0.6663500629815551 on fold 5
--- Fold #6 ---
Traini

In [94]:
target=train_feats["score"]
lgb_oof_pred=models_and_errors_dict['lgbm'][9][4]
cat_oof_pred=models_and_errors_dict['catboost'][9][4]
xgboost_oof_pred=models_and_errors_dict['xgboost'][9][4]
margin=1000
target=target.values
current_RMSE=RMSE(target,(lgb_oof_pred+cat_oof_pred+xgboost_oof_pred)/3)
best_i=0
best_j=0
for i in range(0,margin):
    for j in range(0,margin-i):
        #k=1000-i-j
        blend_oof_pred=(i*lgb_oof_pred+j*cat_oof_pred+(margin-i-j)*xgboost_oof_pred)/margin
        if RMSE(target,blend_oof_pred)<current_RMSE:
            print(f"current_RMSE:{current_RMSE}")
            current_RMSE=RMSE(target,blend_oof_pred)
            best_i=i
            best_j=j

blending_weights['lgbm']=best_i/margin
blending_weights['catboost']=best_j/margin
blending_weights['xgboost']=(margin-best_i-best_j)/margin
print(f"blending_weights:{blending_weights}")

current_RMSE:0.6162508675470676
current_RMSE:0.6162393822443927
current_RMSE:0.6162261591171604
current_RMSE:0.6162129900631403
current_RMSE:0.6161998750857993
current_RMSE:0.6161868141885904
current_RMSE:0.6161738073749525
current_RMSE:0.6161608546483106
current_RMSE:0.6161479560120758
current_RMSE:0.6161351114696452
current_RMSE:0.6161223210244019
current_RMSE:0.6161095846797151
current_RMSE:0.61609690243894
current_RMSE:0.6160842743054179
current_RMSE:0.6160717002824759
current_RMSE:0.6160591803734273
current_RMSE:0.6160467145815716
current_RMSE:0.6160343029101937
current_RMSE:0.6160219453625652
current_RMSE:0.6160096419419431
current_RMSE:0.6159973926515708
current_RMSE:0.6159851974946776
current_RMSE:0.6159730564744788
current_RMSE:0.6159609695941752
current_RMSE:0.6159489368569545
current_RMSE:0.6159369582659896
current_RMSE:0.6159250338244396
current_RMSE:0.6159131635354498
current_RMSE:0.6159013474021511
current_RMSE:0.6158895854276605
current_RMSE:0.615877877615081
current_RMS

In [95]:
y_hats = dict()
test_essays_copy=test_essaysdf.copy()
best_features=train_feats.drop(['score','score_class'],axis=1).keys().values

submission_df = pd.DataFrame(test_feats['id'])
submission_df['score'] = 3.5


X_unseen = test_feats.copy()[best_features]

X_unseen.drop(columns=['id'], inplace=True)

X_unseen.replace([np.inf, -np.inf], np.nan, inplace=True)

for model_name, model_info in models_and_errors_dict.items():
    print(f'\n--- {model_name} ---\n')
    

    X_unseen_copy = X_unseen.copy()
    y_hats[model_name] = []

    for ix, (trained_model, error, imputer, scaler,oof_pred) in enumerate(model_info, start=1):
        print(f"Using model {ix} with error {error}")

        if model_name in model_with_scaled_features:
            X_unseen_imputed = imputer.transform(X_unseen_copy)
            X_unseen_scaled = scaler.transform(X_unseen_imputed)
            y_hats[model_name].append(trained_model.predict(X_unseen_scaled))
        else:
            y_hats[model_name].append(trained_model.predict(X_unseen_copy))
    
    if y_hats[model_name]:
        y_hat_avg = np.mean(y_hats[model_name], axis=0)
        submission_df['score_' + model_name] = y_hat_avg
    print("Done.")
print("blending")
blended_score=np.zeros((len(test_essays_copy)))
for k, v in blending_weights.items():
    blended_score += submission_df['score_' + k] * v
print(f"blended_score:{blended_score}")


--- lgbm ---

Using model 1 with error 0.5915866246197972
Using model 2 with error 0.6229203543715441
Using model 3 with error 0.5799143965420785
Using model 4 with error 0.6370054652138515
Using model 5 with error 0.6581942026875308
Using model 6 with error 0.6663500629815551
Using model 7 with error 0.6167316633325122
Using model 8 with error 0.6054299344112196
Using model 9 with error 0.5923641344954647
Using model 10 with error 0.6699150882878421
Done.

--- catboost ---

Using model 1 with error 0.5855517341761023
Using model 2 with error 0.6070085501747627
Using model 3 with error 0.5753115632523214
Using model 4 with error 0.6175967181566
Using model 5 with error 0.6498335358587458
Using model 6 with error 0.6580954282772558
Using model 7 with error 0.6285694470784462
Using model 8 with error 0.6014761806468116
Using model 9 with error 0.57030374284486
Using model 10 with error 0.6520925841037742
Done.

--- xgboost ---

Using model 1 with error 0.5987595416344031
Using model 2 w

In [96]:
blended_score

0    1.804541
1    1.618949
2    1.611147
Name: score_lgbm, dtype: float64

# **Submission**

In [97]:
submission = pd.DataFrame({'id': test_essays.index, 'score': blended_score.values*0.5+publiclgbm_pred["score"].values*0.5})
submission.to_csv('submission.csv', index=False)
submission

,id,score
0,0000aaaa,1.742340
1,2222bbbb,1.577877
2,4444cccc,1.623538
